## VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("C:/Project/Python-Api-Challenge/Weather/Output/CityWeatherData.csv",encoding="utf-8")
cities.head()


,city,countries,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds
0,mataura,PF,"(-84.15719227970935, -158.1656918138079)",-84.157192,-158.165692,57.02,57.02,47,7.67,34
1,heihe,CN,"(50.59119042704734, 126.85828702042306)",50.591190,126.858287,62.60,62.60,88,13.42,40
2,port elizabeth,ZA,"(-89.15485981491796, 34.65092290242765)",-89.154860,34.650923,73.35,73.35,52,2.37,99
3,micheweni,TZ,"(-5.2445735160357, 41.75967388380866)",-5.244574,41.759674,76.39,76.39,74,7.81,19
4,houma,US,"(28.03013570577255, -90.47056875082696)",28.030136,-90.470569,77.52,78.01,83,4.70,1


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["lat","long"]]

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating = False,max_intensity = maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = cities.loc[(cities["max_temp"] > 70) & (cities["max_temp"] < 80) & (cities["clouds"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df["index"]
narrowed_city_df.head()

,city,countries,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds
0,rikitea,PF,"(-45.37345031256914, -130.4928752836081)",-45.373450,-130.492875,72.61,72.61,85,11.03,0
1,awbari,LY,"(25.471105516103577, 12.006118101579546)",25.471106,12.006118,78.19,78.19,23,10.49,0
2,puerto ayora,EC,"(0.9044591373050821, -105.04826183452225)",0.904459,-105.048262,73.60,73.99,78,11.41,0
3,maneadero,MX,"(29.509116457534248, -119.52804855348822)",29.509116,-119.528049,72.00,72.00,91,1.99,0
4,abu kamal,SY,"(34.91785588785095, 41.12793723959243)",34.917856,41.127937,76.77,76.77,77,5.37,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['lat']
    lng = narrowed_city_df.loc[i]['long']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,city,countries,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds,Hotel Name
0,rikitea,PF,"(-45.37345031256914, -130.4928752836081)",-45.373450,-130.492875,72.61,72.61,85,11.03,0,
1,awbari,LY,"(25.471105516103577, 12.006118101579546)",25.471106,12.006118,78.19,78.19,23,10.49,0,
2,puerto ayora,EC,"(0.9044591373050821, -105.04826183452225)",0.904459,-105.048262,73.60,73.99,78,11.41,0,
3,maneadero,MX,"(29.509116457534248, -119.52804855348822)",29.509116,-119.528049,72.00,72.00,91,1.99,0,
4,abu kamal,SY,"(34.91785588785095, 41.12793723959243)",34.917856,41.127937,76.77,76.77,77,5.37,0,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
#<dt>Country</dt><dd>{countries}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["lat", "long"]]

In [8]:

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in  narrowed_city_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…